Create counterfactual dataset

In [35]:
def parse_data(filename):
    pairs = {}
    with open(filename, 'r') as f:
        for i, line in enumerate(f):
            if i % 2 == 0:  # Check if the line number is odd
                data = eval(line)
                question = data['q']
                key = (data['model_a'], data['model_b'], question)
                swapped_key = (data['model_b'], data['model_a'], question)
                if key not in pairs and swapped_key not in pairs:
                    pairs[key] = data['judge']
                else:
                    if swapped_key in pairs:
                        if pairs[swapped_key] == data['judge'] and data['judge'] != 'tie':
                            print(f"\nAnswer Given: {data['judge']}")
                            print(f"Question: {question}")
                            print(f"First case: {swapped_key[0]} vs {swapped_key[1]}")
                            print(f"Second case: {key[0]} vs {key[1]}")

In [36]:
parse_data('chatbot_counterfactual.json')

In [24]:
import json

def create_counterfactual(line_dict):
    """Create a counterfactual by swapping models, answers, and judge."""
    return {
        "model_a": line_dict["model_b"],
        "model_b": line_dict["model_a"],
        "q": line_dict["q"],
        "a": line_dict["b"],
        "b": line_dict["a"],
        "judge": "model_b" if line_dict["judge"] == "model_a" else "model_a"
    }

In [25]:
def process_file(input_lines):
    """Process each line and create counterfactuals where needed."""
    processed_lines = []
    seen_pairs = set()  # To track which counterfactuals we've already seen
    
    for line in input_lines:
        if not line.strip():  # Skip empty lines
            continue
            
        try:
            data = json.loads(line)
            
            # Create a unique key for this QA pair
            qa_key = (data["q"], min(data["a"], data["b"]), max(data["a"], data["b"]))
            
            # If we haven't seen this pair before, add both original and counterfactual
            if qa_key not in seen_pairs and "judge" in data:
                processed_lines.append(data)  # Original
                counterfactual = create_counterfactual(data)
                processed_lines.append(counterfactual)  # Counterfactual
                seen_pairs.add(qa_key)
            
        except json.JSONDecodeError:
            # Handle lines that aren't valid JSON
            if "...(about" in line or "...(line too long;" in line:
                continue
            processed_lines.append(line)
    
    return processed_lines


In [26]:
with open('data_no_tie copy.jsonl', 'r') as file:
    lines = [next(file) for _ in range(95)]

In [27]:
processed_lines = process_file(lines)

In [29]:
len(processed_lines)

190

In [30]:
with open('data_no_tie_with_counterfactuals.jsonl', 'w') as file:
    for line in processed_lines:
        if isinstance(line, str):
            file.write(line)
        else:
            file.write(json.dumps(line) + '\n')

In [31]:
def calculate_accuracy(ground_truth, predictions):
    if len(ground_truth) != len(predictions):
        raise ValueError("Lists must have the same length")
    
    correct = sum(1 for gt, pred in zip(ground_truth, predictions) if gt == pred)
    total = len(ground_truth)
    accuracy = correct / total
    
    return accuracy

In [33]:
ground_truth = [2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 1, 0, 0, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
predictions =  [0, 1, 2, 2, 0, 2, 2, 0, 2, 0, 2, 2, 0, 2, 0, 0, 0, 2, 2, 0, 0, 2, 0, 0, 2, 0, 2, 0, 2, 2, 2, 1, 2, 2, 2, 2, 2, 0, 0, 2, 0, 0, 2, 2, 2, 0, 1, 1, 2, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 1, 1, 1, 1, 0, 2, 2, 1, 2, 2, 1, 0, 2, 2, 1, 1, 0, 1, 1, 1, 1, 1, 2, 2, 0, 1, 1, 0, 1, 0, 0, 2, 1, 1, 0, 2, 2, 0, 2, 2, 0, 0, 2, 2, 0, 1, 0, 2, 2, 0, 0, 0, 0, 2, 0, 1, 0, 0, 2, 1, 1, 2, 2, 2, 0, 2, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 0, 2, 0, 0, 0, 2, 2, 2, 0, 0, 2, 0, 0, 2, 0, 0, 2, 2, 0, 1, 2, 2, 0, 0, 2, 2, 0, 2, 0, 2, 2, 0, 2, 0, 2, 0, 2, 1, 2, 1, 1, 2, 2, 0, 0, 0, 2, 0, 2]
    # Second list

accuracy = calculate_accuracy(ground_truth, predictions)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.3679


Weighted Circular Evaluation(WCE)

In [37]:
import numpy as np

def calculate_scs(pred_abc, pred_bca, pred_cab):
    """
    Calculate the Switch Consistency Score (SCS).

    Args:
        pred_abc (list): Predictions for ABC configuration.
        pred_bca (list): Predictions for BCA configuration.
        pred_cab (list): Predictions for CAB configuration.

    Returns:
        float: Switch Consistency Score (SCS).
    """
    # Ensure all predictions have the same length
    assert len(pred_abc) == len(pred_bca) == len(pred_cab), "Predictions must be of equal length."

    n = len(pred_abc)
    agreement_scores = []

    for i in range(n):
        predictions = [pred_abc[i], pred_bca[i], pred_cab[i]]
        # Count the most common prediction
        most_common_count = max(predictions.count(0), predictions.count(1), predictions.count(2))
        agreement = most_common_count / 3  # Agreement as a fraction
        agreement_scores.append(agreement)

    # Average agreement scores for all instances
    scs = np.mean(agreement_scores)
    return scs

In [91]:
def calculate_conditional_scs(pred_abc, pred_bca, pred_cab, ground_truth):
    """
    Calculate SCS only for examples where at least one configuration matches ground truth.
    
    Args:
        pred_abc (list): Predictions for ABC configuration
        pred_bca (list): Predictions for BCA configuration
        pred_cab (list): Predictions for CAB configuration
        ground_truth (list): Ground truth labels
        
    Returns:
        float: Conditional SCS score
        float: Percentage of examples used in calculation
    """
    assert len(pred_abc) == len(pred_bca) == len(pred_cab) == len(ground_truth)
    
    n = len(pred_abc)
    agreement_scores = []
    examples_used = 0
    
    for i in range(n):
        predictions = [pred_abc[i], pred_bca[i], pred_cab[i]]
        # print(ground_truth[i], predictions, ground_truth[i] in predictions)
        # Check if any prediction matches ground truth
        if ground_truth[i] in predictions:
            examples_used += 1
            most_common_count = max(predictions.count(0), 
                                  predictions.count(1), 
                                  predictions.count(2))
            agreement = most_common_count / 3
            agreement_scores.append(agreement)
    
    if not agreement_scores:
        return 0.0, 0.0
        
    percentage_used = examples_used / n
    conditional_scs = np.mean(agreement_scores)
    
    return conditional_scs, percentage_used

Normal

In [60]:
gt = [1, 2, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 2, 2, 0, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 1, 0, 0, 2, 0, 0, 2, 1, 0, 0, 2, 2, 1, 2, 2, 2, 2, 2, 0, 0, 1, 2, 2, 1, 2, 1, 1, 0, 2, 2, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 2, 1, 0, 0, 1, 1, 1, 1, 0, 1, 2, 1, 1, 0, 2, 2, 0, 0, 0, 1, 0, 1, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 2, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 2, 0, 2, 2, 0, 0, 1, 1, 1, 0, 1, 0]
# Example data
pred_abc = [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 0, 2, 2, 0, 2, 1, 1, 2, 0, 2, 2, 1, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
pred_bca = [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 1, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
pred_cab = [2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 1, 0, 0, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
pred_bca = [1 if x == 0 else 2 if x == 1 else 0 for x in pred_bca]
pred_cab = [2 if x == 0 else 0 if x == 1 else 1 for x in pred_cab]
print()


In [61]:
# Calculate SCS ZS-CAL
scs = calculate_scs(pred_abc, pred_bca, pred_cab)
print(f"Switch Consistency Score (SCS): {scs:.4f}")

Switch Consistency Score (SCS): 0.3903


In [62]:
cscs, n = calculate_conditional_scs(pred_abc, pred_bca, pred_cab, gt)
print(f"SCS when one correct: {cscs:.4f}\n Correct recall: {n}")

SCS when one correct: 0.3781
 Correct recall: 0.9637305699481865


In [107]:
gt = [1, 2, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 2, 2, 0, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 1, 0, 0, 2, 0, 0, 2, 1, 0, 0, 2, 2, 1, 2, 2, 2, 2, 2, 0, 0, 1, 2, 2, 1, 2, 1, 1, 0, 2, 2, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 2, 1, 0, 0, 1, 1, 1, 1, 0, 1, 2, 1, 1, 0, 2, 2, 0, 0, 0, 1, 0, 1, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 2, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 2, 0, 2, 2, 0, 0, 1, 1, 1, 0, 1, 0]
# Example data
pred_abc = [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0]
pred_bca = [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 0, 1, 1, 0, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 2, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0]
pred_cab = [1, 0, 0, 0, 0, 2, 2, 0, 0, 1, 2, 2, 1, 2, 0, 1, 1, 0, 2, 0, 0, 2, 0, 0, 2, 1, 2, 0, 2, 1, 0, 0, 2, 1, 2, 0, 2, 1, 1, 2, 1, 2, 2, 2, 0, 0, 0, 2, 2, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 2, 2, 2, 0, 2, 0, 0, 2, 2, 2, 0, 1, 0, 0, 2, 1, 2, 2, 2, 2, 2, 0, 0, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 0, 2, 1, 2, 2, 2, 1, 2, 1, 2, 0, 0, 1, 1, 2, 2, 0, 1, 2, 1, 2, 2, 2, 2, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 2, 1, 1, 1, 2, 2, 0, 0, 0, 2, 1, 1, 2, 0, 1, 0, 2, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 2, 0, 1, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0]
pred_bca = [1 if x == 0 else 2 if x == 1 else 0 for x in pred_bca]
pred_cab = [2 if x == 0 else 0 if x == 1 else 1 for x in pred_cab]


In [109]:
print(pred_abc[96], pred_abc[97], pred_bca[96], pred_bca[97],pred_cab[96], pred_cab[97])

1 0 2 0 0 1


In [64]:
# Calculate SCS FS-CAL
scs = calculate_scs(pred_abc, pred_bca, pred_cab)
print(f"Switch Consistency Score (SCS): {scs:.4f}")

Switch Consistency Score (SCS): 0.5250


In [65]:
cscs, n = calculate_conditional_scs(pred_abc, pred_bca, pred_cab, gt)
print(f"SCS when one correct: {cscs:.4f}\n Correct recall: {n}")

SCS when one correct: 0.4990
 Correct recall: 0.844559585492228


First prompt

In [103]:
gt = [1, 2, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 2, 2, 0, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 1, 0, 0, 2, 0, 0, 2, 1, 0, 0, 2, 2, 1, 2, 2, 2, 2, 2, 0, 0, 1, 2, 2, 1, 2, 1, 1, 0, 2, 2, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 2, 1, 0, 0, 1, 1, 1, 1, 0, 1, 2, 1, 1, 0, 2, 2, 0, 0, 0, 1, 0, 1, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 2, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 2, 0, 2, 2, 0, 0, 1, 1, 1, 0, 1, 0]
# Example data
pred_abc = [2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 1, 2, 1, 1, 2, 1, 0, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 1, 1, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 1, 1, 0, 2, 1, 2, 0, 2, 2, 2, 2, 1, 1, 2, 0, 2, 2, 1, 0, 2, 2, 2, 1, 0, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
pred_bca = [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
pred_cab = [1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 1, 1, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 1, 2]
pred_bca = [1 if x == 0 else 2 if x == 1 else 0 for x in pred_bca]
pred_cab = [2 if x == 0 else 0 if x == 1 else 1 for x in pred_cab]


In [104]:
# Calculate SCS ZS-CAL
scs = calculate_scs(pred_abc, pred_bca, pred_cab)
print(f"Switch Consistency Score (SCS): {scs:.4f}")

Switch Consistency Score (SCS): 0.4381


In [105]:
pred_abc = [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 2, 0, 2, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0]
pred_bca = [1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 2, 2, 0, 1, 1, 0, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 2, 2, 1, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0]
pred_cab = [1, 1, 0, 1, 1, 2, 2, 1, 0, 0, 2, 2, 0, 2, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 2, 1, 2, 0, 2, 1, 0, 1, 2, 0, 1, 1, 2, 1, 1, 2, 1, 0, 2, 2, 0, 0, 0, 2, 2, 0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 2, 0, 2, 2, 0, 0, 2, 1, 2, 2, 0, 0, 2, 1, 1, 2, 2, 2, 0, 0, 0, 0, 2, 1, 2, 2, 2, 2, 0, 0, 0, 2, 0, 1, 2, 2, 2, 0, 2, 2, 1, 2, 2, 1, 2, 2, 2, 1, 2, 1, 2, 0, 2, 0, 0, 2, 2, 0, 0, 2, 0, 2, 2, 2, 2, 0, 0, 1, 2, 1, 0, 0, 0, 2, 0, 2, 2, 2, 2, 2, 0, 2, 0, 0, 2, 0, 1, 2, 0, 1, 1, 2, 2, 0, 0, 2, 2, 0, 0, 2, 1, 1, 2, 2, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 1, 1, 2, 0, 1, 1, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 1]
pred_bca = [1 if x == 0 else 2 if x == 1 else 0 for x in pred_bca]
pred_cab = [2 if x == 0 else 0 if x == 1 else 1 for x in pred_cab]


In [106]:
# Calculate SCS FS-CAL
scs = calculate_scs(pred_abc, pred_bca, pred_cab)
print(f"Switch Consistency Score (SCS): {scs:.4f}")

Switch Consistency Score (SCS): 0.5515


Latest prompt

In [100]:
gt = [1, 2, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 2, 2, 0, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 1, 0, 0, 2, 0, 0, 2, 1, 0, 0, 2, 2, 1, 2, 2, 2, 2, 2, 0, 0, 1, 2, 2, 1, 2, 1, 1, 0, 2, 2, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 2, 1, 0, 0, 1, 1, 1, 1, 0, 1, 2, 1, 1, 0, 2, 2, 0, 0, 0, 1, 0, 1, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 2, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 2, 0, 2, 2, 0, 0, 1, 1, 1, 0, 1, 0]

pred_abc = [2, 0, 0, 0, 2, 2, 1, 2, 2, 0, 2, 2, 1, 2, 2, 2, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 0, 2, 0, 0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 0, 2, 0, 0, 0, 0, 1, 0, 2, 0, 0, 2, 2, 0, 2, 0, 2, 2, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 1, 2, 1, 1, 0, 1, 1, 2, 2, 1, 1, 0, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 2, 0, 2, 1, 1, 1, 2, 1, 1, 1, 2, 0, 0, 0, 2, 2, 0, 2, 0, 2, 2, 2, 2, 1, 2, 1, 0, 0, 0, 2, 1, 1, 2, 0, 2, 0, 1, 0, 2, 2, 0, 1, 1, 2, 2, 2, 1, 2, 0, 2, 2, 2, 2, 2, 2, 2, 1]
pred_bca = [2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 2, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 0, 2, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 2, 2, 0, 0, 0, 0, 1, 0, 2, 2, 0, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 2, 2, 1, 2, 1, 1, 0, 1, 1, 2, 2, 1, 2, 0, 2, 2, 0, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 0, 1, 1, 2, 1, 1, 2, 2, 0, 2, 1, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 1, 2, 0, 0, 2, 1, 1, 2, 1, 2, 0, 1, 1, 2, 0, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
pred_cab = [1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 0, 0, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 1, 2, 1, 1, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2]
pred_bca = [1 if x == 0 else 2 if x == 1 else 0 for x in pred_bca]
pred_cab = [2 if x == 0 else 0 if x == 1 else 1 for x in pred_cab]
print(pred_bca)
print(pred_cab)

[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 2, 1, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 2, 0, 2, 2, 1, 2, 2, 0, 0, 2, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 2, 2, 0, 0, 1, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 1, 1, 0, 2, 2, 0, 2, 0, 1, 2, 2, 0, 1, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1,

In [101]:
# Calculate SCS ZS-CAL
scs = calculate_scs(pred_abc, pred_bca, pred_cab)
print(f"Switch Consistency Score (SCS): {scs:.4f}")

Switch Consistency Score (SCS): 0.4897


In [102]:
cscs, n = calculate_conditional_scs(pred_abc, pred_bca, pred_cab, gt)
print(f"SCS when one correct: {cscs:.4f}\n Correct recall: {n}")

SCS when one correct: 0.4603
 Correct recall: 0.865979381443299


In [97]:
gt = [1, 2, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 2, 2, 0, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 1, 0, 0, 2, 0, 0, 2, 1, 0, 0, 2, 2, 1, 2, 2, 2, 2, 2, 0, 0, 1, 2, 2, 1, 2, 1, 1, 0, 2, 2, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 2, 1, 0, 0, 1, 1, 1, 1, 0, 1, 2, 1, 1, 0, 2, 2, 0, 0, 0, 1, 0, 1, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 2, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 2, 0, 2, 2, 0, 0, 1, 1, 1, 0, 1, 0]

pred_abc = [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 2, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]
pred_bca = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
pred_cab = [1, 1, 1, 1, 1, 2, 1, 1, 0, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 2, 0, 2, 2, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 1, 1, 1, 2, 1, 2, 2, 0, 2, 2, 1, 2, 2, 1, 0, 2, 1, 1, 2, 2, 2, 0, 0, 2, 0, 2, 1, 2, 2, 2, 2, 0, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 1, 2, 0, 2, 1, 1, 2, 2, 0, 1, 2, 0, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 1, 1, 2, 1, 1, 2, 2, 1, 1, 1, 0, 1, 0, 1, 2, 0, 0, 0, 2, 1, 1, 2, 1, 1, 1, 2, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1]
pred_bca = [1 if x == 0 else 2 if x == 1 else 0 for x in pred_bca]
pred_cab = [2 if x == 0 else 0 if x == 1 else 1 for x in pred_cab]

In [98]:
# Calculate SCS FS-CAL
scs = calculate_scs(pred_abc, pred_bca, pred_cab)
print(f"Switch Consistency Score (SCS): {scs:.4f}")

Switch Consistency Score (SCS): 0.4880


In [99]:
cscs, n = calculate_conditional_scs(pred_abc, pred_bca, pred_cab, gt)
print(f"SCS when one correct: {cscs:.4f}\n Correct recall: {n}")

SCS when one correct: 0.4603
 Correct recall: 0.865979381443299


Chatbot normal

In [110]:
gt = [2, 2, 0, 1, 2, 2, 0, 1, 2, 2, 0, 1, 2, 2, 0, 1, 1, 0, 0, 1, 2, 2, 1, 0, 1, 0, 0, 1, 2, 2, 0, 1, 0, 1, 2, 2, 1, 0, 0, 1, 2, 2, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 2, 2, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 2, 2, 2, 2, 2, 2, 0, 1, 0, 1, 2, 2, 1, 0, 2, 2, 0, 1, 2, 2, 0, 1, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 0, 1, 2, 2, 2, 2, 2, 2, 0, 1, 1, 0, 2, 2, 1, 0, 1, 0, 2, 2, 1, 0, 2, 2, 2, 2, 0, 1, 2, 2, 1, 0, 2, 2, 1, 0, 2, 2, 2, 2, 0, 1, 0, 1, 2, 2, 2, 2, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 2, 2, 2, 2, 0, 1, 1, 0, 0, 1, 2, 2, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 1, 0, 1, 0, 2, 2, 1, 0, 0, 1, 2, 2, 2, 2, 2, 2, 1, 0, 1, 0, 1, 0, 1, 0, 2, 2, 0, 1, 2, 2, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 0, 1, 1, 0, 2, 2, 1, 0, 0, 1, 0, 1, 1, 0, 2, 2, 2, 2, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 2, 2, 2, 2, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 2, 2, 2, 2, 0, 1, 0, 1, 1, 0, 1, 0, 2, 2, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 2, 2, 2, 2, 0, 1, 2, 2, 1, 0, 0, 1, 0, 1, 2, 2, 2, 2, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 2, 2, 1, 0, 0, 1, 0, 1, 0, 1, 2, 2, 2, 2, 0, 1, 2, 2, 0, 1, 0, 1, 0, 1, 2, 2, 1, 0, 2, 2, 2, 2, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 0, 1, 0, 1, 0, 1, 2, 2, 0, 1, 0, 1, 2, 2, 0, 1, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 0, 1, 1, 0, 1, 0, 2, 2, 0, 1, 2, 2, 0, 1, 2, 2, 0, 1, 0, 1, 2, 2, 2, 2, 0, 1, 0, 1, 2, 2, 0, 1, 2, 2, 1, 0, 1, 0, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 0, 1, 2, 2, 1, 0, 0, 1, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 1, 0, 0, 1, 0, 1, 2, 2, 1, 0, 1, 0, 2, 2, 0, 1, 1, 0, 2, 2, 2, 2, 1, 0, 0, 1, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 0, 1, 1, 0, 2, 2, 1, 0, 2, 2, 0, 1]
# Example data
pred_abc = [0, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 2, 0, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 0, 0, 2, 2, 1, 2, 2, 0, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 0, 2, 2, 2, 2, 2, 1, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 0, 2, 2, 2, 2, 2, 1, 2, 2, 1, 0, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 0, 2, 1, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 0, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 1, 0, 0, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 0, 0, 0, 2, 1, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 1, 2, 2, 2]
pred_bca = [0, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
pred_cab = [2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 0, 1, 1, 2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1]
pred_bca = [1 if x == 0 else 2 if x == 1 else 0 for x in pred_bca]
pred_cab = [2 if x == 0 else 0 if x == 1 else 1 for x in pred_cab]


In [111]:
# Calculate SCS ZS-CAL
scs = calculate_scs(pred_abc, pred_bca, pred_cab)
print(f"Switch Consistency Score (SCS): {scs:.4f}")

Switch Consistency Score (SCS): 0.4256


In [112]:
gt = [2, 2, 0, 1, 2, 2, 0, 1, 2, 2, 0, 1, 2, 2, 0, 1, 1, 0, 0, 1, 2, 2, 1, 0, 1, 0, 0, 1, 2, 2, 0, 1, 0, 1, 2, 2, 1, 0, 0, 1, 2, 2, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 2, 2, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 2, 2, 2, 2, 2, 2, 0, 1, 0, 1, 2, 2, 1, 0, 2, 2, 0, 1, 2, 2, 0, 1, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 0, 1, 2, 2, 2, 2, 2, 2, 0, 1, 1, 0, 2, 2, 1, 0, 1, 0, 2, 2, 1, 0, 2, 2, 2, 2, 0, 1, 2, 2, 1, 0, 2, 2, 1, 0, 2, 2, 2, 2, 0, 1, 0, 1, 2, 2, 2, 2, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 2, 2, 2, 2, 0, 1, 1, 0, 0, 1, 2, 2, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 1, 0, 1, 0, 2, 2, 1, 0, 0, 1, 2, 2, 2, 2, 2, 2, 1, 0, 1, 0, 1, 0, 1, 0, 2, 2, 0, 1, 2, 2, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 0, 1, 1, 0, 2, 2, 1, 0, 0, 1, 0, 1, 1, 0, 2, 2, 2, 2, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 2, 2, 2, 2, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 2, 2, 2, 2, 0, 1, 0, 1, 1, 0, 1, 0, 2, 2, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 2, 2, 2, 2, 0, 1, 2, 2, 1, 0, 0, 1, 0, 1, 2, 2, 2, 2, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 2, 2, 1, 0, 0, 1, 0, 1, 0, 1, 2, 2, 2, 2, 0, 1, 2, 2, 0, 1, 0, 1, 0, 1, 2, 2, 1, 0, 2, 2, 2, 2, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 0, 1, 0, 1, 0, 1, 2, 2, 0, 1, 0, 1, 2, 2, 0, 1, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 0, 1, 1, 0, 1, 0, 2, 2, 0, 1, 2, 2, 0, 1, 2, 2, 0, 1, 0, 1, 2, 2, 2, 2, 0, 1, 0, 1, 2, 2, 0, 1, 2, 2, 1, 0, 1, 0, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 0, 1, 2, 2, 1, 0, 0, 1, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 1, 0, 0, 1, 0, 1, 2, 2, 1, 0, 1, 0, 2, 2, 0, 1, 1, 0, 2, 2, 2, 2, 1, 0, 0, 1, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 0, 1, 1, 0, 2, 2, 1, 0, 2, 2, 0, 1]
# Example data
pred_abc = [1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0]
pred_bca = [1, 0, 0, 2, 2, 1, 1, 0, 0, 1, 2, 1, 1, 2, 1, 1, 1, 0, 2, 2, 1, 0, 1, 0, 1, 1, 1, 1, 0, 2, 0, 1, 2, 2, 0, 0, 1, 1, 2, 2, 1, 1, 1, 1, 0, 2, 2, 2, 2, 2, 1, 1, 2, 2, 0, 1, 2, 2, 2, 0, 1, 0, 1, 1, 2, 0, 2, 0, 2, 0, 0, 2, 1, 0, 0, 2, 0, 2, 1, 0, 1, 0, 2, 2, 1, 1, 0, 1, 2, 2, 2, 2, 1, 1, 2, 0, 2, 2, 2, 2, 2, 1, 2, 0, 1, 1, 1, 1, 2, 2, 0, 0, 0, 2, 0, 1, 2, 2, 2, 0, 1, 1, 2, 1, 1, 1, 2, 0, 1, 0, 0, 0, 2, 2, 2, 2, 2, 1, 1, 1, 2, 1, 0, 1, 1, 0, 2, 2, 2, 1, 0, 2, 2, 1, 0, 1, 1, 1, 0, 1, 2, 1, 0, 2, 2, 1, 1, 1, 1, 0, 2, 2, 1, 1, 1, 1, 2, 1, 0, 2, 2, 2, 2, 0, 0, 2, 2, 1, 1, 0, 2, 0, 2, 2, 1, 0, 0, 1, 2, 2, 1, 0, 2, 1, 0, 1, 2, 0, 2, 2, 1, 1, 0, 1, 1, 1, 1, 1, 2, 0, 0, 0, 2, 1, 2, 0, 1, 0, 2, 2, 2, 2, 1, 1, 0, 2, 0, 2, 1, 0, 1, 2, 0, 1, 0, 0, 1, 0, 1, 2, 0, 2, 1, 1, 0, 1, 2, 0, 0, 1, 0, 2, 1, 1, 2, 1, 2, 2, 0, 0, 1, 1, 0, 2, 2, 1, 2, 2, 2, 2, 0, 1, 1, 0, 2, 2, 0, 2, 2, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 2, 2, 0, 2, 2, 0, 1, 1, 1, 0, 2, 1, 1, 1, 2, 2, 0, 2, 2, 1, 0, 2, 0, 2, 2, 0, 1, 1, 2, 1, 1, 0, 0, 2, 1, 1, 0, 2, 2, 0, 1, 2, 0, 1, 0, 0, 0, 1, 0, 0, 1, 2, 0, 1, 0, 0, 2, 0, 0, 2, 1, 1, 0, 2, 1, 2, 1, 1, 1, 2, 0, 1, 1, 2, 1, 0, 2, 1, 2, 2, 1, 1, 2, 2, 0, 1, 1, 0, 2, 1, 2, 0, 1, 1, 1, 0, 0, 2, 2, 2, 2, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 2, 2, 2, 0, 1, 0, 1, 2, 2, 0, 0, 1, 1, 0, 2, 1, 1, 0, 2, 2, 1, 1, 0, 1, 2, 1, 2, 0, 2, 2, 2, 0, 0, 2, 2, 1, 2, 1, 0, 0, 1, 1, 0, 1, 0, 2, 1, 1, 1, 2, 0, 2, 0, 2, 0, 1, 0, 1, 2, 2, 1, 1, 0, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 0, 0, 1, 2, 2, 0, 2, 0, 1, 2, 0, 0, 2, 1, 2, 0, 1, 0, 1, 0, 2, 2, 1, 2, 2, 2, 1, 1, 1, 1, 0, 2, 2, 0, 0, 1, 1, 0, 2, 0, 2, 0, 1, 0, 0, 2, 1, 0, 0, 2, 0, 2, 0, 2, 2, 1, 2, 0, 2, 2, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 2, 0, 1, 0, 1, 2, 2, 2, 0, 1, 1, 1, 2, 0, 0, 2, 2, 2, 0, 2, 2, 2, 1, 0, 2, 1, 1, 1, 1, 2, 2, 0, 2, 2, 2, 1, 0, 1]
pred_cab = [2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 0, 0, 0, 2, 0, 2, 0, 2, 2, 0, 2, 0, 2, 1, 2, 0, 1, 1, 0, 0, 2, 0, 0, 2, 2, 0, 2, 0, 2, 0, 2, 0, 0, 2, 0, 2, 2, 0, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 0, 1, 1, 2, 0, 2, 0, 0, 2, 0, 2, 2, 0, 0, 2, 2, 0, 2, 0, 0, 2, 0, 0, 0, 2, 2, 0, 2, 2, 2, 1, 0, 2, 0, 2, 2, 2, 2, 2, 2, 1, 2, 0, 2, 1, 2, 0, 0, 2, 0, 0, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 0, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 0, 0, 2, 2, 0, 2, 0, 2, 0, 0, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 0, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 0, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 1, 2, 0, 2, 2, 2, 0, 1, 1, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 0, 0, 2, 2, 0, 0, 2, 0, 2, 2, 2, 2, 2, 0, 2, 0, 1, 1, 0, 2, 2, 0, 2, 0, 1, 1, 2, 0, 0, 2, 2, 0, 2, 2, 1, 2, 2, 2, 0, 2, 1, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2, 0, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2, 0, 2, 2, 0, 2, 0, 0, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 0, 0, 0, 2, 2, 1, 0, 2, 2, 0, 0, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 0, 2, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 0, 2, 0, 2, 0, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 2, 0, 0, 2, 2, 0, 2, 2, 2, 2, 0, 2, 0, 2, 2, 0, 2, 2, 2, 0, 0, 2, 0, 0, 2, 2, 0, 2, 2, 0, 2, 0, 0, 2, 0, 2, 0, 2, 0, 2, 2, 0, 0, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 1, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 0, 2, 2, 2, 2, 0, 0, 2, 0, 2, 0, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 0, 2, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2, 1, 2, 0, 0, 2, 0, 2, 0, 2, 2, 0, 0, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 0, 2, 1, 0, 2, 0, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 0, 0, 2, 0, 2, 2, 2, 2, 0, 0, 2, 2, 0, 0, 2, 0, 2, 2, 0, 2, 0, 2, 0, 0, 2, 0, 2, 0, 2, 0, 0, 0, 2, 0, 2, 2, 2, 0, 2, 0, 0]
pred_bca = [1 if x == 0 else 2 if x == 1 else 0 for x in pred_bca]
pred_cab = [2 if x == 0 else 0 if x == 1 else 1 for x in pred_cab]


In [113]:
# Calculate SCS FS-CAL
scs = calculate_scs(pred_abc, pred_bca, pred_cab)
print(f"Switch Consistency Score (SCS): {scs:.4f}")

Switch Consistency Score (SCS): 0.5594


enhanced 1

In [114]:
gt = [2, 2, 0, 1, 2, 2, 0, 1, 2, 2, 0, 1, 2, 2, 0, 1, 1, 0, 0, 1, 2, 2, 1, 0, 1, 0, 0, 1, 2, 2, 0, 1, 0, 1, 2, 2, 1, 0, 0, 1, 2, 2, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 2, 2, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 2, 2, 2, 2, 2, 2, 0, 1, 0, 1, 2, 2, 1, 0, 2, 2, 0, 1, 2, 2, 0, 1, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 0, 1, 2, 2, 2, 2, 2, 2, 0, 1, 1, 0, 2, 2, 1, 0, 1, 0, 2, 2, 1, 0, 2, 2, 2, 2, 0, 1, 2, 2, 1, 0, 2, 2, 1, 0, 2, 2, 2, 2, 0, 1, 0, 1, 2, 2, 2, 2, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 2, 2, 2, 2, 0, 1, 1, 0, 0, 1, 2, 2, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 1, 0, 1, 0, 2, 2, 1, 0, 0, 1, 2, 2, 2, 2, 2, 2, 1, 0, 1, 0, 1, 0, 1, 0, 2, 2, 0, 1, 2, 2, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 0, 1, 1, 0, 2, 2, 1, 0, 0, 1, 0, 1, 1, 0, 2, 2, 2, 2, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 2, 2, 2, 2, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 2, 2, 2, 2, 0, 1, 0, 1, 1, 0, 1, 0, 2, 2, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 2, 2, 2, 2, 0, 1, 2, 2, 1, 0, 0, 1, 0, 1, 2, 2, 2, 2, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 2, 2, 1, 0, 0, 1, 0, 1, 0, 1, 2, 2, 2, 2, 0, 1, 2, 2, 0, 1, 0, 1, 0, 1, 2, 2, 1, 0, 2, 2, 2, 2, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 0, 1, 0, 1, 0, 1, 2, 2, 0, 1, 0, 1, 2, 2, 0, 1, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 0, 1, 1, 0, 1, 0, 2, 2, 0, 1, 2, 2, 0, 1, 2, 2, 0, 1, 0, 1, 2, 2, 2, 2, 0, 1, 0, 1, 2, 2, 0, 1, 2, 2, 1, 0, 1, 0, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 0, 1, 2, 2, 1, 0, 0, 1, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 1, 0, 0, 1, 0, 1, 2, 2, 1, 0, 1, 0, 2, 2, 0, 1, 1, 0, 2, 2, 2, 2, 1, 0, 0, 1, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 0, 1, 1, 0, 2, 2, 1, 0, 2, 2, 0, 1]
# Example data
pred_abc = [0, 0, 2, 2, 2, 2, 2, 2, 0, 0, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 0, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 0, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 0, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 2, 1, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 0, 0, 0, 2, 1, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2]
pred_bca = [2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
pred_cab = [2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1]
pred_bca = [1 if x == 0 else 2 if x == 1 else 0 for x in pred_bca]
pred_cab = [2 if x == 0 else 0 if x == 1 else 1 for x in pred_cab]


In [115]:
# Calculate SCS ZS-CAL
scs = calculate_scs(pred_abc, pred_bca, pred_cab)
print(f"Switch Consistency Score (SCS): {scs:.4f}")

Switch Consistency Score (SCS): 0.4039


In [116]:
gt = [2, 2, 0, 1, 2, 2, 0, 1, 2, 2, 0, 1, 2, 2, 0, 1, 1, 0, 0, 1, 2, 2, 1, 0, 1, 0, 0, 1, 2, 2, 0, 1, 0, 1, 2, 2, 1, 0, 0, 1, 2, 2, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 2, 2, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 2, 2, 2, 2, 2, 2, 0, 1, 0, 1, 2, 2, 1, 0, 2, 2, 0, 1, 2, 2, 0, 1, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 0, 1, 2, 2, 2, 2, 2, 2, 0, 1, 1, 0, 2, 2, 1, 0, 1, 0, 2, 2, 1, 0, 2, 2, 2, 2, 0, 1, 2, 2, 1, 0, 2, 2, 1, 0, 2, 2, 2, 2, 0, 1, 0, 1, 2, 2, 2, 2, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 2, 2, 2, 2, 0, 1, 1, 0, 0, 1, 2, 2, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 1, 0, 1, 0, 2, 2, 1, 0, 0, 1, 2, 2, 2, 2, 2, 2, 1, 0, 1, 0, 1, 0, 1, 0, 2, 2, 0, 1, 2, 2, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 0, 1, 1, 0, 2, 2, 1, 0, 0, 1, 0, 1, 1, 0, 2, 2, 2, 2, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 2, 2, 2, 2, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 2, 2, 2, 2, 0, 1, 0, 1, 1, 0, 1, 0, 2, 2, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 2, 2, 2, 2, 0, 1, 2, 2, 1, 0, 0, 1, 0, 1, 2, 2, 2, 2, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 2, 2, 1, 0, 0, 1, 0, 1, 0, 1, 2, 2, 2, 2, 0, 1, 2, 2, 0, 1, 0, 1, 0, 1, 2, 2, 1, 0, 2, 2, 2, 2, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 0, 1, 0, 1, 0, 1, 2, 2, 0, 1, 0, 1, 2, 2, 0, 1, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 0, 1, 1, 0, 1, 0, 2, 2, 0, 1, 2, 2, 0, 1, 2, 2, 0, 1, 0, 1, 2, 2, 2, 2, 0, 1, 0, 1, 2, 2, 0, 1, 2, 2, 1, 0, 1, 0, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 0, 1, 2, 2, 1, 0, 0, 1, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 1, 0, 0, 1, 0, 1, 2, 2, 1, 0, 1, 0, 2, 2, 0, 1, 1, 0, 2, 2, 2, 2, 1, 0, 0, 1, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 0, 1, 1, 0, 2, 2, 1, 0, 2, 2, 0, 1]
# Example data
pred_abc = [1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 2, 2, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 2, 2, 1, 0, 0, 0]
pred_bca = [1, 0, 0, 2, 2, 0, 0, 0, 0, 1, 2, 2, 0, 2, 2, 2, 2, 0, 0, 2, 1, 0, 0, 0, 2, 2, 1, 1, 0, 1, 0, 2, 0, 2, 0, 0, 0, 1, 0, 2, 2, 2, 1, 1, 0, 2, 2, 2, 2, 2, 0, 1, 2, 0, 0, 1, 1, 1, 2, 0, 1, 0, 2, 1, 2, 0, 2, 0, 2, 0, 0, 2, 1, 0, 0, 2, 0, 2, 2, 0, 2, 0, 2, 2, 2, 2, 0, 1, 2, 2, 1, 2, 1, 1, 2, 0, 0, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 1, 2, 2, 0, 0, 0, 2, 0, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 2, 1, 0, 2, 1, 0, 0, 2, 0, 2, 2, 2, 2, 0, 2, 2, 0, 0, 1, 0, 2, 1, 1, 2, 0, 0, 2, 2, 0, 0, 0, 1, 0, 2, 2, 0, 1, 0, 1, 2, 1, 0, 2, 0, 2, 2, 0, 0, 2, 2, 0, 1, 0, 2, 2, 2, 2, 1, 0, 0, 2, 0, 2, 0, 0, 1, 1, 0, 1, 2, 0, 0, 2, 2, 2, 0, 0, 0, 2, 0, 2, 2, 2, 0, 0, 2, 0, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 0, 0, 0, 2, 0, 2, 0, 0, 1, 0, 0, 2, 0, 2, 0, 0, 0, 1, 2, 0, 0, 1, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 1, 1, 0, 2, 2, 1, 2, 2, 2, 2, 0, 1, 1, 0, 2, 2, 0, 2, 2, 0, 0, 2, 1, 2, 0, 1, 0, 0, 1, 0, 2, 2, 0, 2, 2, 0, 0, 0, 1, 0, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 0, 1, 0, 2, 2, 0, 0, 0, 2, 0, 1, 0, 0, 1, 1, 0, 0, 2, 2, 0, 0, 2, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 1, 1, 0, 2, 2, 2, 0, 0, 2, 2, 0, 1, 0, 0, 1, 0, 2, 2, 2, 2, 1, 1, 2, 2, 0, 1, 2, 2, 1, 1, 2, 0, 1, 1, 0, 0, 0, 2, 2, 2, 2, 0, 2, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 2, 0, 2, 2, 1, 0, 1, 0, 0, 2, 2, 0, 0, 1, 1, 0, 2, 1, 1, 2, 2, 2, 2, 1, 0, 0, 1, 0, 2, 2, 2, 0, 2, 0, 0, 2, 2, 0, 2, 0, 0, 0, 2, 2, 0, 1, 0, 1, 0, 0, 0, 2, 0, 2, 0, 2, 0, 1, 0, 2, 2, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 2, 2, 0, 0, 1, 2, 2, 0, 2, 0, 1, 2, 2, 0, 2, 2, 0, 0, 1, 0, 1, 0, 2, 2, 0, 2, 2, 2, 1, 0, 2, 1, 0, 0, 2, 0, 0, 1, 1, 0, 1, 0, 2, 0, 1, 0, 0, 2, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 1, 0, 2, 2, 1, 1, 0, 2, 0, 2, 0, 0, 2, 2, 2, 1, 2, 0, 1, 2, 0, 2, 2, 0, 2, 0, 2, 0, 0, 1, 0, 2, 0, 0, 0, 2, 2, 2, 0, 2, 0, 1, 1, 0, 0]
pred_cab = [1, 0, 0, 1, 1, 2, 1, 0, 2, 2, 1, 1, 1, 1, 2, 1, 1, 2, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 2, 1, 0, 2, 1, 1, 0, 2, 1, 2, 1, 2, 0, 1, 0, 2, 1, 1, 2, 0, 1, 0, 1, 1, 2, 1, 1, 1, 0, 1, 1, 1, 2, 1, 0, 1, 0, 1, 1, 1, 1, 2, 2, 1, 0, 2, 1, 0, 1, 2, 2, 1, 0, 1, 1, 1, 2, 1, 2, 1, 0, 2, 2, 1, 2, 0, 1, 1, 2, 1, 2, 1, 0, 1, 2, 0, 1, 1, 0, 2, 2, 2, 1, 0, 2, 1, 1, 0, 1, 0, 1, 2, 1, 1, 2, 0, 1, 2, 2, 0, 1, 1, 0, 2, 1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 1, 0, 2, 2, 2, 1, 1, 2, 2, 0, 0, 1, 1, 2, 2, 2, 2, 2, 1, 0, 2, 1, 0, 0, 1, 2, 2, 1, 2, 1, 0, 1, 1, 1, 1, 1, 1, 2, 0, 0, 1, 1, 1, 2, 1, 0, 1, 2, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 0, 1, 0, 0, 2, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 2, 1, 1, 0, 2, 1, 1, 2, 2, 1, 2, 0, 0, 2, 2, 0, 1, 2, 1, 2, 2, 2, 2, 1, 1, 1, 0, 1, 2, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 2, 2, 0, 2, 1, 2, 0, 1, 1, 0, 1, 2, 1, 1, 1, 1, 0, 1, 2, 1, 0, 1, 2, 2, 1, 0, 0, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 0, 0, 0, 1, 0, 0, 1, 1, 2, 1, 1, 0, 2, 0, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 0, 1, 1, 2, 1, 0, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 1, 1, 2, 1, 0, 1, 1, 1, 0, 0, 1, 2, 2, 2, 2, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 0, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 0, 1, 0, 0, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 2, 0, 1, 2, 2, 1, 2, 0, 0, 2, 2, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 2, 1, 2, 1, 0, 1, 1, 2, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 2, 1, 2, 2, 2, 1, 1, 1, 0, 0, 0, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 0, 2, 1, 2, 2, 1, 1, 0, 1, 2, 1, 0, 1, 1, 0, 1, 1, 2, 2, 1, 1, 1, 2, 1, 2, 2, 1, 2, 2, 2, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 2, 2, 2, 0, 1, 2, 2, 2, 1, 1, 1, 1, 2, 2, 1, 2, 2, 0, 1, 1, 2, 1, 0, 1, 1, 0, 1, 1, 2, 1, 2, 2, 2, 1, 2, 0, 1]
pred_bca = [1 if x == 0 else 2 if x == 1 else 0 for x in pred_bca]
pred_cab = [2 if x == 0 else 0 if x == 1 else 1 for x in pred_cab]


In [117]:
# Calculate SCS FS-CAL
scs = calculate_scs(pred_abc, pred_bca, pred_cab)
print(f"Switch Consistency Score (SCS): {scs:.4f}")

Switch Consistency Score (SCS): 0.6356


enhanced 2

In [118]:
gt = [2, 2, 0, 1, 2, 2, 0, 1, 2, 2, 0, 1, 2, 2, 0, 1, 1, 0, 0, 1, 2, 2, 1, 0, 1, 0, 0, 1, 2, 2, 0, 1, 0, 1, 2, 2, 1, 0, 0, 1, 2, 2, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 2, 2, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 2, 2, 2, 2, 2, 2, 0, 1, 0, 1, 2, 2, 1, 0, 2, 2, 0, 1, 2, 2, 0, 1, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 0, 1, 2, 2, 2, 2, 2, 2, 0, 1, 1, 0, 2, 2, 1, 0, 1, 0, 2, 2, 1, 0, 2, 2, 2, 2, 0, 1, 2, 2, 1, 0, 2, 2, 1, 0, 2, 2, 2, 2, 0, 1, 0, 1, 2, 2, 2, 2, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 2, 2, 2, 2, 0, 1, 1, 0, 0, 1, 2, 2, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 1, 0, 1, 0, 2, 2, 1, 0, 0, 1, 2, 2, 2, 2, 2, 2, 1, 0, 1, 0, 1, 0, 1, 0, 2, 2, 0, 1, 2, 2, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 0, 1, 1, 0, 2, 2, 1, 0, 0, 1, 0, 1, 1, 0, 2, 2, 2, 2, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 2, 2, 2, 2, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 2, 2, 2, 2, 0, 1, 0, 1, 1, 0, 1, 0, 2, 2, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 2, 2, 2, 2, 0, 1, 2, 2, 1, 0, 0, 1, 0, 1, 2, 2, 2, 2, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 2, 2, 1, 0, 0, 1, 0, 1, 0, 1, 2, 2, 2, 2, 0, 1, 2, 2, 0, 1, 0, 1, 0, 1, 2, 2, 1, 0, 2, 2, 2, 2, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 0, 1, 0, 1, 0, 1, 2, 2, 0, 1, 0, 1, 2, 2, 0, 1, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 0, 1, 1, 0, 1, 0, 2, 2, 0, 1, 2, 2, 0, 1, 2, 2, 0, 1, 0, 1, 2, 2, 2, 2, 0, 1, 0, 1, 2, 2, 0, 1, 2, 2, 1, 0, 1, 0, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 0, 1, 2, 2, 1, 0, 0, 1, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 1, 0, 0, 1, 0, 1, 2, 2, 1, 0, 1, 0, 2, 2, 0, 1, 1, 0, 2, 2, 2, 2, 1, 0, 0, 1, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 0, 1, 1, 0, 2, 2, 1, 0, 2, 2, 0, 1]
# Example data
pred_abc = [0, 0, 0, 2, 0, 0, 2, 1, 0, 0, 1, 1, 1, 2, 0, 0, 0, 2, 0, 0, 0, 0, 1, 1, 2, 1, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 2, 2, 1, 1, 0, 0, 0, 0, 0, 2, 2, 2, 1, 1, 2, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 2, 0, 0, 2, 0, 2, 2, 2, 2, 0, 0, 1, 1, 0, 0, 0, 0, 2, 2, 1, 1, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 1, 1, 2, 2, 1, 1, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 1, 1, 2, 2, 2, 2, 0, 0, 2, 2, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 0, 2, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 0, 0, 2, 2, 1, 1, 2, 2, 0, 0, 0, 2, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 1, 0, 0, 0, 0, 2, 2, 0, 0, 2, 2, 0, 1, 1, 1, 2, 1, 2, 0, 0, 0, 2, 2, 2, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 2, 2, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 2, 2, 0, 0, 2, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 2, 1, 0, 2, 0, 0, 0, 0, 2, 2, 2, 1, 0, 2, 1, 1, 0, 0, 0, 2, 2, 2, 2, 2, 2, 0, 0, 0, 1, 2, 1, 1, 2, 2, 2, 0, 0, 0, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 2, 2, 2, 2, 1, 2, 2, 2, 2, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 2, 1, 1, 0, 0, 1, 1, 0, 0, 2, 2, 2, 0, 2, 2, 0, 2, 0, 0, 2, 0, 2, 2, 2, 1, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 2, 0, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 0, 0, 2, 0, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 2, 0, 2, 2, 0, 0, 1, 2, 2, 1, 1, 2, 2, 2, 2, 0, 2, 2, 0, 1, 2, 1, 1, 1, 2, 2, 0, 0, 2, 2, 0, 2, 0, 0, 2, 0, 0, 0, 2, 2, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 0, 2, 0, 0, 0, 0, 2, 2, 1, 1, 0, 2, 0, 0, 2, 2, 0, 0, 1, 1, 2, 2, 0, 0, 2, 2, 2, 2, 2, 1, 2, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 2, 2, 0, 2, 2, 0, 0, 2, 2, 1, 1, 1, 0, 2, 0, 2, 0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 1]
pred_bca = [0, 0, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 1, 2, 2, 2, 0, 2, 2, 2, 0, 2, 1, 1, 2, 1, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 1, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 1, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 0, 2, 2, 0, 0, 2, 2, 2, 1, 0, 0, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 1, 1, 2, 2, 2, 0, 1, 2, 0, 2, 2, 2, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 1, 1, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 1, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 0, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 1, 1, 2, 2, 2, 2]
pred_cab = [0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 1, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
pred_bca = [1 if x == 0 else 2 if x == 1 else 0 for x in pred_bca]
pred_cab = [2 if x == 0 else 0 if x == 1 else 1 for x in pred_cab]


In [119]:
# Calculate SCS ZS-CAL
scs = calculate_scs(pred_abc, pred_bca, pred_cab)
print(f"Switch Consistency Score (SCS): {scs:.4f}")

Switch Consistency Score (SCS): 0.5161


In [120]:
gt = [2, 2, 0, 1, 2, 2, 0, 1, 2, 2, 0, 1, 2, 2, 0, 1, 1, 0, 0, 1, 2, 2, 1, 0, 1, 0, 0, 1, 2, 2, 0, 1, 0, 1, 2, 2, 1, 0, 0, 1, 2, 2, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 2, 2, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 2, 2, 2, 2, 2, 2, 0, 1, 0, 1, 2, 2, 1, 0, 2, 2, 0, 1, 2, 2, 0, 1, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 0, 1, 2, 2, 2, 2, 2, 2, 0, 1, 1, 0, 2, 2, 1, 0, 1, 0, 2, 2, 1, 0, 2, 2, 2, 2, 0, 1, 2, 2, 1, 0, 2, 2, 1, 0, 2, 2, 2, 2, 0, 1, 0, 1, 2, 2, 2, 2, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 2, 2, 2, 2, 0, 1, 1, 0, 0, 1, 2, 2, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 1, 0, 1, 0, 2, 2, 1, 0, 0, 1, 2, 2, 2, 2, 2, 2, 1, 0, 1, 0, 1, 0, 1, 0, 2, 2, 0, 1, 2, 2, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 0, 1, 1, 0, 2, 2, 1, 0, 0, 1, 0, 1, 1, 0, 2, 2, 2, 2, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 2, 2, 2, 2, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 2, 2, 2, 2, 0, 1, 0, 1, 1, 0, 1, 0, 2, 2, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 2, 2, 2, 2, 0, 1, 2, 2, 1, 0, 0, 1, 0, 1, 2, 2, 2, 2, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 2, 2, 1, 0, 0, 1, 0, 1, 0, 1, 2, 2, 2, 2, 0, 1, 2, 2, 0, 1, 0, 1, 0, 1, 2, 2, 1, 0, 2, 2, 2, 2, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 0, 1, 0, 1, 0, 1, 2, 2, 0, 1, 0, 1, 2, 2, 0, 1, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 0, 1, 1, 0, 1, 0, 2, 2, 0, 1, 2, 2, 0, 1, 2, 2, 0, 1, 0, 1, 2, 2, 2, 2, 0, 1, 0, 1, 2, 2, 0, 1, 2, 2, 1, 0, 1, 0, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 0, 1, 2, 2, 1, 0, 0, 1, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 1, 0, 0, 1, 0, 1, 2, 2, 1, 0, 1, 0, 2, 2, 0, 1, 1, 0, 2, 2, 2, 2, 1, 0, 0, 1, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 0, 1, 1, 0, 2, 2, 1, 0, 2, 2, 0, 1]
# Example data
pred_abc = [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 2, 2, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 2, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 1, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 2, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 2, 1, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 2, 2, 1, 0, 0, 0]
pred_bca = [1, 1, 0, 2, 2, 1, 1, 0, 0, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 0, 1, 0, 2, 1, 1, 1, 0, 1, 0, 2, 0, 2, 0, 0, 1, 1, 1, 2, 2, 1, 1, 1, 0, 2, 1, 2, 0, 1, 1, 1, 2, 2, 0, 1, 1, 1, 2, 0, 1, 0, 0, 1, 1, 0, 2, 0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 2, 1, 0, 1, 1, 0, 2, 1, 1, 0, 1, 0, 2, 2, 0, 1, 1, 2, 0, 0, 2, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 0, 2, 1, 1, 1, 1, 0, 1, 1, 2, 2, 0, 1, 1, 2, 0, 1, 1, 2, 0, 1, 1, 0, 0, 2, 2, 2, 0, 2, 1, 1, 1, 2, 1, 0, 1, 2, 0, 0, 2, 0, 1, 0, 2, 2, 1, 0, 1, 1, 1, 0, 2, 2, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 2, 1, 0, 2, 0, 2, 2, 1, 0, 2, 2, 1, 1, 1, 2, 1, 1, 2, 1, 0, 0, 1, 1, 2, 0, 0, 1, 2, 0, 1, 2, 0, 0, 2, 1, 1, 0, 1, 1, 1, 1, 1, 2, 2, 0, 0, 1, 1, 2, 0, 1, 1, 2, 2, 2, 2, 1, 1, 0, 2, 0, 2, 0, 0, 1, 2, 0, 2, 0, 0, 1, 0, 1, 2, 0, 2, 1, 0, 0, 1, 2, 0, 1, 1, 0, 1, 1, 1, 1, 1, 2, 2, 0, 0, 0, 1, 0, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 0, 1, 2, 0, 2, 2, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 2, 0, 2, 2, 0, 1, 1, 1, 0, 1, 0, 1, 1, 2, 1, 1, 2, 2, 2, 0, 2, 0, 2, 1, 0, 1, 1, 2, 1, 1, 0, 0, 1, 1, 1, 0, 2, 2, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 2, 1, 1, 1, 1, 1, 2, 0, 1, 0, 0, 1, 1, 1, 1, 1, 2, 1, 1, 0, 2, 0, 1, 1, 1, 2, 0, 1, 1, 1, 1, 1, 0, 1, 2, 1, 0, 2, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 2, 2, 0, 0, 1, 0, 0, 2, 1, 1, 0, 2, 2, 1, 2, 0, 1, 1, 1, 2, 1, 2, 1, 2, 1, 0, 2, 2, 0, 2, 0, 0, 0, 1, 1, 0, 1, 0, 2, 1, 1, 0, 2, 0, 2, 0, 1, 0, 1, 0, 1, 2, 0, 1, 1, 0, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 0, 1, 1, 2, 2, 0, 2, 0, 2, 2, 0, 0, 2, 1, 1, 1, 1, 0, 1, 0, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 0, 1, 2, 0, 0, 1, 1, 0, 2, 0, 2, 0, 1, 1, 0, 2, 0, 0, 0, 2, 0, 2, 0, 2, 1, 1, 2, 1, 2, 2, 0, 0, 1, 1, 1, 0, 1, 0, 2, 1, 2, 1, 0, 1, 0, 0, 0, 1, 2, 1, 0, 0, 1, 1, 1, 2, 0, 0, 2, 0, 0, 0, 2, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 0, 2, 2, 2, 1, 1, 1]
pred_cab = [1, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 0, 0, 2, 2, 0, 1, 2, 1, 2, 2, 2, 2, 1, 2, 0, 2, 1, 2, 0, 1, 1, 2, 2, 2, 0, 2, 2, 2, 0, 2, 1, 2, 0, 2, 1, 2, 2, 1, 1, 1, 0, 2, 1, 0, 2, 0, 2, 2, 2, 1, 2, 1, 2, 0, 2, 2, 0, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 0, 0, 2, 1, 2, 1, 2, 2, 0, 2, 2, 2, 1, 1, 2, 0, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 1, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 1, 0, 2, 2, 2, 2, 0, 2, 0, 0, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 0, 1, 1, 0, 2, 1, 2, 0, 0, 1, 2, 0, 2, 2, 1, 2, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 1, 1, 2, 2, 0, 0, 1, 2, 2, 1, 2, 0, 2, 2, 1, 2, 2, 0, 2, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 2, 0, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 0, 2, 1, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1, 2, 1, 1, 1, 2, 2, 2, 2, 2, 0, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, 0, 0, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 0, 2, 1, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 2, 0, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2, 0, 2, 1, 1, 2, 2, 2, 0, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 0, 1, 2, 2, 1, 1, 1, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 0, 2, 2, 0, 2, 1, 0, 2, 0, 2, 1, 2, 1, 2, 2, 0, 0, 2, 0, 2, 1, 1, 2, 2, 2, 1, 2, 1, 1, 1, 2, 1, 2, 2, 2, 1, 2, 2, 2, 0, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 0, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 2, 2, 2, 2, 0, 1, 1, 2, 2, 0, 2, 2, 2, 1, 2, 2, 2, 0, 2, 2, 0, 2, 0, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 0, 1, 1, 2, 2, 1, 2, 0, 0, 1, 2, 1, 2, 2, 1, 1, 2, 0, 1, 1, 2, 2, 1, 2, 0, 2, 0, 2, 1, 2, 1, 1, 1, 1, 1, 1, 0, 2, 2, 2]
pred_bca = [1 if x == 0 else 2 if x == 1 else 0 for x in pred_bca]
pred_cab = [2 if x == 0 else 0 if x == 1 else 1 for x in pred_cab]


In [121]:
# Calculate SCS FS-CAL
scs = calculate_scs(pred_abc, pred_bca, pred_cab)
print(f"Switch Consistency Score (SCS): {scs:.4f}")

Switch Consistency Score (SCS): 0.5400
